<a href="https://colab.research.google.com/github/Firojpaudel/Machine-Learning-Notes/blob/main/Practical%20Deep%20Learning%20For%20Coders/Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training a Digit Classifier

In [1]:
#First initializing the notebook
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.2 MB/s eta 0:00:00
Mounted at /content/gdrive


In [3]:
from fastai.vision.all import *
from fastbook import *

### **Lesson 1:** How images are represented in a computer:

For the initial learning, I'm just creating a model that classifies any image as *3 or 7*. Before that, downloading a sample of MNIST that contains images of just these digits

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
## Checking the directory
path.ls()

In [ ]:
## Checking what's inside the training set
(path/'train').ls()

So, yeah! there are images of 7 and 3 in the training set. The images inside these folders are labelled accordingly.

In [ ]:
## Taking a look in these folders to check the images.
three = (path/'train'/'3').ls().sorted()
seven= (path/'train'/'7').ls().sorted()

In [ ]:
three

In [ ]:
seven

They are all full of .png files.


In [ ]:
## Taking a look at one of them
img3 = Image.open(three[0])
img3

In computer as we know all data is represented as a number, we have to convert the images into an array or tensor before model training.

In [ ]:
## Converting to array
array(img3)

'''
This is similar to doing:
array = np.array(img3)
array

'''

In [ ]:
#@ Side learnings:
#@ If we wish to see the array from particular index in both rows and columns, we could do like:
np.array(img3)[4:10,4:10]

In [ ]:
## Now, in pytorch, it's kinda different. We have tensors instead of arrays
tensor(img3)[4:10, 4:10]

Also, we could do sth like color coding in pandas.

In [ ]:
#@ Visualizing in pandas
df= pd.DataFrame(tensor(img3)[3:13, 9:19])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

### **First Try:** Pixel Similarity


* Step1: Would be to get the average of pixel values for each of our two groups

In [ ]:
#@ Creating tensors containg a single image
seven_tensors = [tensor(Image.open(o)) for o in seven]
three_tensors = [tensor(Image.open(o)) for o in three]

#@ Checking the lengths of these created tensors
len(three_tensors), len(seven_tensors)

In [ ]:
#! Just checking the tensor:
show_image(three_tensors[4])

In [ ]:
#@ Now stacking
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255

#@ Checking the Shape
stacked_threes.shape

**The meaning of this output:**<br>
It's trying to say that we have 6,131 images, each of size 28×28 pixels.


<details>
  <summary>
    <b>The reason behind converting to float and dividing by 255</b>
  </summary>
- Some of the operations in PyTorch, such as taking means, require us to cast our integer types into float types. So, converting the stacks to float as well.<br>
- When converted to float, the pixel values are also expected to be in between 0 to 1. So dividing by 255.  
</details>

In [ ]:
#@ Gettig the rank of the tensor
len(stacked_threes.shape)

'''
This is similar to doing:
stacked_threes.shape
'''


##### **Ranks defined**:
\- Here's what the rank of tensors mean:
  - **Rank 0**= Scalar, *real number or constant*
  - **Rank 1**= Vector, *can be represente as list of values*
  - **Rank 2**= Matrix, *to represent 2D data structures*
  - **Rank 3**= 3D Tensor, *cube or stack of matrices*
  - **Rank 4**= 4D Tensor, *to represent images in batch form: where the 4th dimension reprents the batch size*




In [ ]:
#@ Now computing the mean and looking at what we should be expecting for peak image for 3 (for classification)
mean3 = stacked_threes.mean(0)
show_image(mean3)

In [ ]:
#@ Doing the same for seven
mean7 = stacked_sevens.mean(0)
show_image(mean7)

In [ ]:
## Just picking an arbitary 3s or 7s and seeing how far are we from the "ideal digits"
arb_3 = stacked_threes[1]
show_image(arb_3)

arb_7 = stacked_sevens[1]
show_image(arb_7)

<details>
  <summary>
    Measuring the distances:
  </summary>
- Take the mean of absolute value of differences. Also called as <i>L1 Norm</i> <br>
- Then find the Root Mean Sqaured Error (RMSE). Also called as <i>L2 Norm</i>
</details>

In [ ]:
#@ Implementing the above information

dist_3_abs = (arb_3 - mean3).abs().mean() #L1 Norm
dist_3_sqr = ((arb_3 - mean3)**2).mean().sqrt() #L2 Norm
dist_3_abs, dist_3_sqr

In [ ]:
dist_7_abs= (arb_7 - mean7).abs().mean()
dist_7_sqr= ((arb_7 - mean7)**2).mean().sqrt()
dist_7_abs, dist_7_sqr

In [ ]:
#@ Now implementing the same using loss functions.
import torch.nn.functional as F

F.l1_loss(arb_3.float(), mean3), F.mse_loss(arb_3, mean3).sqrt() #l1 stands for L1 Norm and mse is just L2 Norm

##### Numpy Arrays and PyTorch tensors

So, this part it's just a revision about what Numpy arrays and tensors are.

<details>
  <summary>
    <b>Why use Numpy arrays?</b>
  </summary>
Since they are built using optimized C code, Numpy arrays are way way faster than the python code.
</details>

<details>
  <summary>
    <b>Why PyTorch Tensors?</b>
  </summary>
They do the same task as Numpy array. Just more specified with GPUs.
</details>

In [ ]:
#@ Showing in code:
data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
tnsr = torch.tensor(data)
ary= np.array(data)

In [ ]:
tnsr

In [ ]:
ary

In [ ]:
#@ Performing just basic drills:
print(f"Tensor's shape: {tnsr.shape}")
print(f"Accessing 1st row of tensor: {tnsr[0]}")
print(f"Accessing 2st column of tensor: {tnsr[:, 1]}")

In [ ]:
#!! More drillings further:
print("Sliced displays:")
print(f"Accessing the 1st row 2nd column value: {tnsr[0, 1]}")
print(f"Accessing the 1st rows 2nd and 3rd column values: {tnsr[0, 1:3]}") #Here, for a:b the output would be only till a to b-1

In [ ]:
#@ Just lookig at the tensor type
print(f"The tensor's type: {tnsr.type()}")
print(f"The datatype of this tensor: {tnsr.dtype}")

In [ ]:
##@ Comuptations in tensor is simple:
print(f"The mean of tensor is: {tnsr.float().mean()}") #As mentioned earlier, means can't be calculated until and unless its float datatype
print(f"Multiplying the tensor with 0.01 we get: {tnsr*0.01}")

##### Computing Metrics Using Broadcasting

In [ ]:
## Creating the valid set tensors
valid_3_tnsr = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tnsr = valid_3_tnsr.float()/255
valid_7_tnsr = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tnsr = valid_7_tnsr.float()/255

#@ Checking their shapes
print(f"Shape of valid 3s: {valid_3_tnsr.shape}")
print(f"Shape of valid 7s: {valid_7_tnsr.shape}")

In [ ]:
## Function for calculating MAE:
def mnist_distance(a, b):
    return (a-b).abs().mean((-1, -2)) #!! Tells PyTorch that we want to take the mean ranging over values indexed by last two axes of tensor.

mnist_distance(arb_3, mean3)

In [ ]:
#@ Implementing MAE on the valid sets:
valid_3_dist = mnist_distance(valid_3_tnsr, mean3)
valid_3_dist, valid_3_dist.shape

<details>
  <summary>
    <b>The concept of broadcasting </b>
  </summary>
Broadcasting is a powerful feature in PyTorch that allows you to perform operations on tensors of different shapes without explicitly reshaping them. Here's a breakdown of how it works:<br>
When we perform an operation between two tensors of different ranks (dimensions), PyTorch automatically expands the smaller tensor to match the shape of the larger tensor. <i>This process is called broadcasting</i>. It simplifies tensor operations and makes the code more concise.
</details>

In [ ]:
#@ Simple example to illustrate broadcasting:

##! Performing calculations in normal operations:
a = torch.tensor([10., 6, -4]) #Tensor of shape 3
b = torch.tensor([2., 8, 7])   #Tensor of shape 3
print("The normal operation a+b:", a + b)

##! Changing the shape of tensor a:
a= torch.tensor(1)
print("After changing the shape of tensor a: ", a.ndim)
print("We got the rank of a to be scalar")
print("Checking the rank of unchanged tensor b: ", b.ndim)
print("And b is of rank 1 which is vector.")

So theoritically, we should not be able to perform operations between scalars and vectors. **But that's where broadcasting kicks in.**

In [ ]:
## Seeing broadcasting in action:
res = a+b #Broadcasting in the backend of PyTorch
print("So adding two gives:",res)

<details>
  <summary>
      <b>Why is this effecient anyway? </b>
  </summary>
Broadcasting is efficient because PyTorch doesn't actually copy the smaller tensor multiple times. Instead, it pretends the tensor has the larger shape and performs the operation in a highly optimized manner, often in C or CUDA for GPUs. This makes tensor operations both expressive and performant.
</details>

In [ ]:
#@ Using this concept to figure out whether an image is 3 by using the following logic:
def is_3(x):
    return mnist_distance(x, mean3) < mnist_distance(x, mean7)

print(is_3(arb_3), is_3(arb_3).float())

print("The float value of 1.0 here means True")

In [ ]:
#@ Thanks to broadcasting, we can also test it on the full validation set of 3s
is_3(valid_3_tnsr)

In [ ]:
## Now calculating accuracies
accuracy_3= is_3(valid_3_tnsr).float().mean()
accuracy_7= is_3(valid_7_tnsr).float().mean()
accuracy_3, accuracy_7

So, for the function `is_3()` the `valid_3_tenr` gave the probability value closer to 1 meaning the accuracy is valid. Now testig the same for `def is_7()`

In [ ]:
#@ Back to 7
def is_7(x):
    return mnist_distance(x, mean7) < mnist_distance(x, mean3)

is_7(arb_7), is_7(arb_7).float()

In [ ]:
accuracy_7s = is_7(valid_7_tnsr).float().mean()
accuracy_7s

In [ ]:
#@ But turns out we don't need to test this way:
#@ If we know the concept of probability, the sum total is always 1. So we could do like
accuracy_7 = (1 - is_3(valid_7_tnsr).float()).mean()
accuracy_7

### **Second Try**: SGD

Instead of trying to find the similarity between an image and an "ideal image", we could look at each individual pixel and come up with a set of weights for each, such that the highest weights are associated with those pixels most likely be black for a particular category.

In [ ]:
#@ First going with a simpler most basic approach:

def f(x):                     #!! Initializing the function
  return x**2

#!! Calculating gradients
xt = tensor(3.).requires_grad_()
yt= f(xt)
print(yt)
yt.backward()                 #!! back prop  ---> Calculating the derivative of each layer
xt.grad
print("The value of backprop is:",xt.grad)

So, what we get here is the derivation with respect to xt. i.e.,
$\frac{\partial y}{\partial x} =\frac{\partial x^2}{\partial x} = 2x$

In [ ]:
#@ Now doing the same for vector arguments in the function
xt= tensor([1., 2., 3.], requires_grad=True)
print("The vector arg xt: ",xt)


#!! Then we add sum to our function so that the function can take vectors (rank1 tensor) and return scalar
def f(x):
  return (x**2).sum()

yt = f(xt)
yt
print(yt)
yt.backward()
xt.grad
print("The values become:", xt.grad)

To convert the function to machinne learning classifier, we perform the following steps:

1.   Initialize the weights.
2.   List item
